# Meal Price Calculator

This notebook calculates meal prices based on ingredient costs and a lookup table, and tracks price history for each meal.

In [1]:
# 1. Import Required Libraries
import pandas as pd
import os
from datetime import datetime

In [2]:
# 2. Load Ingredient Prices Data
ingredients_path = '../meal_planning/ingredients.xlsx'
ingredients_df = pd.read_excel(ingredients_path)
# Ensure columns: IngredientID, Latest price, Bonus price, Normal price
for col in ['Bonus price', 'Normal price']:
    if col not in ingredients_df.columns:
        ingredients_df[col] = None
ingredients_df = ingredients_df[['IngredientID', 'FullName','Latest price', 'Bonus price', 'Normal price']].copy()
ingredients_df = ingredients_df.rename(columns={'Latest price': 'Current price'})
print(ingredients_df.head())

  IngredientID                                  FullName  Current price  \
0       ING001           Zanetti Parmigiano reggiano 30+           4.99   
1       ING002                        AH Pecorino romano           6.99   
2       ING003  Natuurfarm de Boed Mais-scharreleieren L           3.89   
3       ING004        Meester & Zn. Gerookte spekreepjes           3.29   
4       ING005                        Heinz Tomato Frito           2.99   

   Bonus price  Normal price  
0          NaN          4.99  
1          NaN          6.99  
2          NaN          3.89  
3          NaN          3.29  
4          NaN          2.99  


In [3]:
# 3. Load Meals-Ingredients Lookup Data
meals_ingredients_path = '../meal_planning/meals_ingredients.xlsx'
meals_ingredients_df = pd.read_excel(meals_ingredients_path)
meals_df = pd.read_excel('../meal_planning/meals.xlsx')
# Ensure columns: MealID, IngredientID, Quantity
meals_ingredients_df = meals_ingredients_df.copy()
print(meals_ingredients_df.head())

  MealID     Gerecht ProductID ProductName IngredientID  Ingredient  Quantity  \
0   M001       Carbo   PROD001  Parmegiano       ING001  Parmegiano      0.33   
1   M001       Carbo   PROD002    Pecorino       ING002    Pecorino      0.33   
2   M001       Carbo   PROD003          Ei       ING003          Ei      0.33   
3   M001       Carbo   PROD004     Spekjes       ING004     Spekjes      1.00   
4   M002  Vodkapasta   PROD005       Frito       ING005       Frito      1.00   

                                   FullName  
0           Zanetti Parmigiano reggiano 30+  
1                        AH Pecorino romano  
2  Natuurfarm de Boed Mais-scharreleieren L  
3        Meester & Zn. Gerookte spekreepjes  
4                        Heinz Tomato Frito  


In [4]:
# 4. Recalculate Meal Price History for Each Week
import numpy as np

# Load historical ingredient prices (should have columns: IngredientID, Date, WeekNr, Current price, Bonus price, Normal price)
ingredients_history_path = '../meal_planning/Ingredients_history.xlsx'
ingredients_history_df = pd.read_excel(ingredients_history_path)

# Ensure WeekNr is present and correct
ingredients_history_df['Date'] = pd.to_datetime(ingredients_history_df['Date'])
ingredients_history_df['WeekNr'] = ingredients_history_df['Date'].dt.isocalendar().week

# Get all unique weeks
all_weeks = np.sort(ingredients_history_df['WeekNr'].unique())

# Prepare to collect all weeks' meal prices
history_records = []

for week in all_weeks:
    week_ingredients = ingredients_history_df[ingredients_history_df['WeekNr'] == week].copy()
    # Use only the latest price per ingredient for the week
    week_ingredients = week_ingredients.sort_values('Date').drop_duplicates('IngredientID', keep='last')
    # Prepare columns to match main calculation
    week_ingredients = week_ingredients[['IngredientID', 'FullName','Latest price', 'Bonus price', 'Normal price','Date']]
    # Merge with meals_ingredients
    merged_df = pd.merge(meals_ingredients_df, week_ingredients, on='IngredientID', how='left')
    merged_df['IngredientCost_Current'] = (merged_df['Quantity'] * merged_df['Latest price']).astype(float).round(2)
    merged_df['IngredientCost_Current'] = merged_df['IngredientCost_Current'].fillna(0)
    merged_df['IngredientSavings'] = merged_df.apply(
        lambda row: round((row['Normal price'] - row['Latest price']) * row['Quantity'], 2)
        if pd.notnull(row['Bonus price']) and pd.notnull(row['Normal price']) and row['Bonus price'] not in [None, 'Not found', 'Error'] and row['Normal price'] not in [None, 'Not found', 'Error']
        else 0.00,
        axis=1
    )
    for col in ['Bonus price', 'Normal price']:
        merged_df[col] = merged_df[col].where(pd.notnull(merged_df[col]), None)
    # For each MealID, ProductID: pick the ingredient variant (IngredientID) with the lowest IngredientCost_Current
    if not merged_df.empty:
        idx = merged_df.groupby(['MealID', 'ProductID'])['IngredientCost_Current'].idxmin()
        cheapest_merged_df = merged_df.loc[idx].copy()
    else:
        cheapest_merged_df = merged_df.copy()
    # List out the ingredients used for each meal (comma separated)
    ingredient_names_col = 'FullName' if 'FullName' in cheapest_merged_df.columns else 'IngredientID'
    if not cheapest_merged_df.empty:
        ingredients_used = (
            cheapest_merged_df.groupby(['MealID', 'Gerecht'])[ingredient_names_col]
            .apply(lambda x: ', '.join(map(str, x)))
            .reset_index()
            .rename(columns={ingredient_names_col: 'IngredientsUsed'})
        )
    else:
        ingredients_used = pd.DataFrame(columns=['MealID', 'Gerecht', 'IngredientsUsed'])
    # Sum for each MealID and Gerecht
    if not cheapest_merged_df.empty:
        meal_prices = cheapest_merged_df.groupby(['MealID', 'Gerecht'], as_index=False).agg({
            'IngredientCost_Current': 'sum',
            'IngredientSavings': 'sum'
        })
    else:
        meal_prices = pd.DataFrame(columns=['MealID', 'Gerecht', 'MealPrice_Current', 'Savings abs'])
    meal_prices = meal_prices.rename(columns={'IngredientCost_Current': 'MealPrice_Current', 'IngredientSavings': 'Savings abs'})
    meal_prices['MealPrice_Current'] = meal_prices['MealPrice_Current'].astype(float).round(2)
    meal_prices['Savings abs'] = meal_prices['Savings abs'].astype(float).round(2)
    meal_prices['MealPrice_Normal'] = (meal_prices['MealPrice_Current'] + meal_prices['Savings abs']).round(2)
    meal_prices['Savings %'] = meal_prices.apply(
        lambda row: round((row['Savings abs'] / row['MealPrice_Normal'] * 100), 2) if row['MealPrice_Normal'] > 0 else None,
        axis=1
    )
    if not meal_prices.empty and not ingredients_used.empty:
        meal_prices = pd.merge(meal_prices, ingredients_used, on=['MealID', 'Gerecht'], how='left')
    else:
        meal_prices['IngredientsUsed'] = None
    # Join with meals.xlsx to get the 'Porties' (portions) column
    meal_prices = pd.merge(meal_prices, meals_df[['MealID', 'Porties']], on='MealID', how='left')
    meal_prices['Price_per_portion'] = meal_prices.apply(
        lambda row: round(row['MealPrice_Current'] / row['Porties'], 2) if pd.notnull(row['Porties']) and row['Porties'] > 0 else None,
        axis=1
    )
    meal_prices['WeekNr'] = week
    # Add the date for the week (use the latest date in the week)
    week_date = week_ingredients['Date'].max() if 'Date' in week_ingredients.columns else None
    meal_prices['Date'] = week_date.strftime('%d-%m-%Y')
    history_records.append(meal_prices)

# Concatenate all weeks
history_df = pd.concat(history_records, ignore_index=True)

# Save to Excel
meal_prices_history_path = '../meal_planning/meal_prices_history.xlsx'
history_df.to_excel(meal_prices_history_path, index=False)
print(f"Meal price history recalculated and saved to {meal_prices_history_path}")

Meal price history recalculated and saved to ../meal_planning/meal_prices_history.xlsx
